In [9]:
import pandas as pd
from src.utils import get_kfold_data
from src.constants import *

In [10]:
data = pd.read_csv(DATA_PATH)

In [11]:
data.head()

,outcome,carat,cut,color,clarity,depth,table,price,x,y,...,a6,a7,a8,a9,a10,b6,b7,b8,b9,b10
0,-26.701232,1.14,Ideal,G,VS1,62.3,56.0,7948,6.73,6.70,...,0.168836,-0.273758,1.107832,1.247795,0.482344,0.489511,-0.321138,0.573382,0.446871,-1.990581
1,6.548093,0.38,Premium,H,VS2,60.5,59.0,898,4.69,4.66,...,-0.256549,0.315373,-0.030326,-0.114335,-1.059588,-1.761360,-1.343951,-1.002550,-0.225030,-0.446653
2,6.612562,0.50,Very Good,E,SI1,60.7,58.0,1351,5.09,5.13,...,-1.193327,-0.657307,-0.591726,-0.446856,-0.765286,-0.816544,-1.397794,-0.477130,0.810509,1.725131
3,-5.073562,0.70,Premium,D,SI1,61.2,58.0,2512,5.74,5.70,...,-1.740788,-1.778860,-0.825070,0.444932,1.173109,0.453606,-0.263440,0.246210,-0.850503,-0.412950
4,-14.436557,0.83,Ideal,G,SI2,62.4,54.0,2751,6.01,6.08,...,-0.859322,1.409268,0.861992,1.109063,-1.436722,-1.461618,0.081787,0.258087,0.851146,2.204813


Inspecting columns

In [12]:
# Find columns
all_columns = data.columns.tolist()
print(all_columns)

numeric_columns = data.select_dtypes(include=["number"]).columns.tolist()
numeric_columns.remove("outcome") # Remove the target column
print(numeric_columns)

non_numeric_columns = data.select_dtypes(exclude=["number"]).columns.tolist()
print(non_numeric_columns)

['outcome', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'a1', 'a2', 'a3', 'a4', 'a5', 'b1', 'b2', 'b3', 'b4', 'b5', 'a6', 'a7', 'a8', 'a9', 'a10', 'b6', 'b7', 'b8', 'b9', 'b10']
['cut', 'color', 'clarity']


In [13]:
for non_numeric_column in non_numeric_columns:
    print(data[non_numeric_column].value_counts())

cut
Ideal        4040
Premium      2439
Very Good    2296
Good          925
Fair          300
Name: count, dtype: int64
color
G    2120
E    1873
F    1746
H    1506
D    1246
I     983
J     526
Name: count, dtype: int64
clarity
SI1     2408
VS2     2256
SI2     1743
VS1     1503
VVS2     951
VVS1     675
IF       318
I1       146
Name: count, dtype: int64


Converting non-numeric features to numerical features

In [14]:
colours = data["color"].value_counts().index.tolist()
print(colours)

for colour in colours:
    data[f"colour_{colour}"] = data["color"].apply(lambda x: 1 if x == colour else 0)
data.drop("color", axis=1, inplace=True)

quality_mappings = {"Ideal": 0, "Premium": 1, "Very Good": 2, "Good": 3, "Fair": 4}
data["cut"] = data["cut"].map(quality_mappings)

clarity_mappings = {"IF": 0, "VVS1": 1, "VVS2": 2, "VS1": 3, "VS2": 4, "SI1": 5, "SI2": 6, "I1": 7}
data["clarity"] = data["clarity"].map(clarity_mappings)
print(data)

['G', 'E', 'F', 'H', 'D', 'I', 'J']
        outcome  carat  cut  clarity  depth  table  price     x     y     z  \
0    -26.701232   1.14    0        3   62.3   56.0   7948  6.73  6.70  4.18   
1      6.548093   0.38    1        4   60.5   59.0    898  4.69  4.66  2.83   
2      6.612562   0.50    2        5   60.7   58.0   1351  5.09  5.13  3.10   
3     -5.073562   0.70    1        5   61.2   58.0   2512  5.74  5.70  3.50   
4    -14.436557   0.83    0        6   62.4   54.0   2751  6.01  6.08  3.77   
...         ...    ...  ...      ...    ...    ...    ...   ...   ...   ...   
9995  10.718277   0.33    0        3   62.6   57.0   1002  4.42  4.40  2.76   
9996 -12.246698   1.01    4        5   69.5   55.0   4853  6.00  5.94  4.15   
9997  11.122516   0.52    2        6   57.9   61.0   1273  5.28  5.33  3.07   
9998 -24.730782   0.31    0        0   62.0   54.0    801  4.35  4.39  2.71   
9999   8.735755   0.37    2        5   59.9   59.0    649  4.68  4.70  2.81   

      ...      

In [15]:
kfold_data = get_kfold_data(data=data, k=NUM_FOLDS, reproducibility_seed=REPRODUCIBILITY_SEED)

Fold: 0/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 1/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 2/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 3/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%

Fold: 4/5
Train shape: (6400, 37) | 64.00%
Validation shape: (1600, 37) | 16.00%
Test shape: (2000, 37) | 20.00%



In [16]:
for fold in range(NUM_FOLDS):
    fold_data = kfold_data[fold]

    train_data = fold_data["train"]
    val_data = fold_data["val"]
    test_data = kfold_data[fold]["test"]

    train_y = train_data["outcome"]
    val_y = val_data["outcome"]
    test_y = test_data["outcome"]

    train_x = train_data.drop(columns=["outcome"])
    val_x = val_data.drop(columns=["outcome"])
    test_x = test_data.drop(columns=["outcome"])

    print(f"Fold {fold+1}/{NUM_FOLDS}")
    print(f"Train data shape: {train_x.shape} | Train target shape: {train_y.shape}")
    print(f"Val data shape: {val_x.shape} | Val target shape: {val_y.shape}")
    print(f"Test data shape: {test_x.shape} | Test target shape: {test_y.shape}")
    print()

Fold 1/5
Train data shape: (6400, 36) | Train target shape: (6400,)
Val data shape: (1600, 36) | Val target shape: (1600,)
Test data shape: (2000, 36) | Test target shape: (2000,)

Fold 2/5
Train data shape: (6400, 36) | Train target shape: (6400,)
Val data shape: (1600, 36) | Val target shape: (1600,)
Test data shape: (2000, 36) | Test target shape: (2000,)

Fold 3/5
Train data shape: (6400, 36) | Train target shape: (6400,)
Val data shape: (1600, 36) | Val target shape: (1600,)
Test data shape: (2000, 36) | Test target shape: (2000,)

Fold 4/5
Train data shape: (6400, 36) | Train target shape: (6400,)
Val data shape: (1600, 36) | Val target shape: (1600,)
Test data shape: (2000, 36) | Test target shape: (2000,)

Fold 5/5
Train data shape: (6400, 36) | Train target shape: (6400,)
Val data shape: (1600, 36) | Val target shape: (1600,)
Test data shape: (2000, 36) | Test target shape: (2000,)

